In [11]:
import pandas as pd
pd.set_option('max_colwidth', 500)
pd.set_option('max_column', 50)

In [12]:
from psaw import PushshiftAPI

api = PushshiftAPI()

In [13]:
import datetime as dt 

start = int(dt.datetime(2020, 1, 1).timestamp())
end = int(dt.datetime(2021, 1, 1).timestamp())

In [14]:
api_request_generator = api.search_submissions(subreddit='News', after = start, before=end, score = ">1")

In [15]:
x = pd.DataFrame([submission.d_ for submission in api_request_generator])

In [16]:
x['date'] = pd.to_datetime(x['created_utc'], utc=True, unit='s')

In [17]:
news = x[['title', 'date', 'score', 'num_comments', 'subreddit']].sort_values(by=['date']).reset_index().drop(columns=['index'])
news

,title,date,score,num_comments,subreddit
0,U-Haul to Implement Nicotine-Free Hiring Policy in 21 States,2020-01-02 18:01:47+00:00,134,518,news
1,FDA issues ban on some flavored vaping products,2020-01-02 18:29:42+00:00,28,43,news
2,Family kicked off plane because daughter threw up before take off forced to cancel vacation,2020-01-02 18:54:03+00:00,30,114,news
3,Kansas police apologize for faked story of expletive on cup,2020-01-02 19:02:08+00:00,745,298,news
4,Lebanese lawyers want Ghosn prosecuted over Israel trip,2020-01-02 19:03:04+00:00,2,2,news
...,...,...,...,...,...
9366,"Louisville police move to fire officers in Breonna Taylor case who got warrant, fired fatal shot",2020-12-30 05:44:28+00:00,2,1,news
9367,"A Black jazz musician's son was falsely accused of stealing an iPhone, father says. The woman accusing him is speaking out to CNN",2020-12-30 17:44:14+00:00,2,39,news
9368,Apple and TikTok remove app used to arrange parties during Covid,2020-12-30 19:32:36+00:00,2,28,news
9369,Most prolific serial killer in US history dies in California,2020-12-31 01:35:25+00:00,2,4,news
